# Cost Calculator

Data is fetched from google sheets: https://docs.google.com/spreadsheets/d/1x4YdgXZi56wqUUObbQAWpH_StFCEkWmMwspQzh5Xu6I/edit?usp=sharing


In [10]:
from __future__ import division
from os import set_inheritable
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import numpy as np
import random
from gsheetcoms import GSH
import pandas as pd
import time as timetracker

In [11]:
##########################################
# Default Values
#########################################
_WORKBOOK = 'RoutingModel'
_TEMP_SKU_DICT = {}

##########################################
# Googel Sheet Communication / Fetch Tables as Dataframes
#########################################
_GSH = GSH()
print('Fetching data tables: 1 of 5 ...', end ="\r")
_ASSUMPTIONS = _GSH.generateDataframe(_WORKBOOK, 'Assumptions')
print('Fetching data tables: 2 of 5 ...', end ="\r")
_ORDERS = _GSH.generateDataframe(_WORKBOOK, 'Orders')
print('Fetching data tables: 3 of 5 ...' , end ="\r")
_SKUS = _GSH.generateDataframe(_WORKBOOK, 'SKUs')
print('Fetching data tables: 4 of 5 ...', end ="\r")
_ORDERS_SKUS = _GSH.generateDataframe(_WORKBOOK, 'Orders_SKUs')
print('Fetching data tables: 5 of 5 ...', end ="\r")
_LAYOUT_ASSUMPTIONS = _GSH.generateDataframe(_WORKBOOK, 'Layout_Assumptions')
print("")
print('Data Fetched Successfully')

##########################################
# General Assumptions
#########################################
print('Creating Global Variables ...')
_STABILIZE_TIME = float(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Stabilize Time'].iloc[0]['Value'])
_PICKING_TIME = float(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Picking Time'].iloc[0]['Value'])
_PICKER_SPEED = float(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Picker Speed'].iloc[0]['Value'])
_PICKER_COST = float(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Picker Cost'].iloc[0]['Value'])
_STABILIZE_TIME = float(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Stabilize Time'].iloc[0]['Value'])
_PICKING_SETUP_TIME = float(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Picking Setup Time'].iloc[0]['Value'])
_PALLET_WIDTH = float(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Pallet Width'].iloc[0]['Value'])
_PALLET_LENGTH = float(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Pallet Length'].iloc[0]['Value'])
_MAX_PALLET_VOLUME = float(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Maximum Pallet Volume'].iloc[0]['Value'])
_CORRIDOR_WIDTH = float(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Corridor Width'].iloc[0]['Value'])
_TRANSPORTATION_COST = float(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Transportation Cost'].iloc[0]['Value'])
_FORKLIFT_TIME_UPDOWN = float(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Forklift Time'].iloc[0]['Value'])
_FORKLIFT_COST = float(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Forklift Operator Cost'].iloc[0]['Value'])

print('Data is ready!')

Initializing Google Spreadheet Connector: gsheet.json
Fetching data tables: 5 of 5 ...
Data Fetched Successfully
Creating Global Variables ...
Data is ready!


In [12]:
def cellInfo(cell_id):
    cell_id = str(cell_id)
    info = {
        'ID' : cell_id,
        'SKU_ID' : _LAYOUT.loc[_LAYOUT['Cell_ID'] == cell_id].iloc[0]['SKU_ID'],
        'x'  : _LAYOUT.loc[_LAYOUT['Cell_ID'] == cell_id].iloc[0]['X'],
        'y'  : _LAYOUT.loc[_LAYOUT['Cell_ID'] == cell_id].iloc[0]['Y'],
        'direction'  : _LAYOUT.loc[_LAYOUT['Cell_ID'] == cell_id].iloc[0]['Direction']
    }
    return info

def distance_calculator(cell1, cell2):
    distance = 0
    pos1 = cellInfo(cell1)
    pos2 = cellInfo(cell2)
    # Start Point
    x1 = int(pos1['x'])
    y1 = int(pos1['y'])
    direction1 = int(pos1['direction'])
    # End Point
    x2 = int(pos2['x'])
    y2 = int(pos2['y'])
    direction2 = int(pos2['direction'])
    
    # Distance y-axis
    distance_y = abs(y2 - y1)

    # Distance x-axis
    distance_x = 0
    x_low = 1
    x_passage = int(_X_MIDDLE)
    x_limit = int(_X_LIMIT)

    # If locations are in same corridor:
    if y1 == y2:
        distance_x = abs(x2 - x1)
    else:
        if direction1 == 1:
            if x1 <= x_passage:
                if direction1 == direction2:
                    if x2 <= x_passage:
                        distance_x = x_passage - x1 + x_passage + x2
                    else:
                        distance_x = x2 - x1
                else:
                    if x2 <= x_passage:
                        distance_x = x_passage - x1 + x_passage - x2
                    else:
                        distance_x = x_limit - x1 + x_limit - x2
            else:
                if direction1 == direction2:
                    if x2 <= x_passage:
                        distance_x = x_limit - x1 + x_limit + x2
                    else:
                        distance_x = x_limit - x1 + x2
                else:
                    distance_x = x_limit - x1 + x_limit - x2
        else:
            if x1 <= x_passage:
                if direction1 == direction2:
                    if x2 <= x_passage:
                        distance_x = x1 + x_passage + x_passage - x2

                    else:
                        distance_x = x1 + x_limit + x_limit - x2
                else:
                        distance_x = x1 + x2
            else:
                if direction1 == direction2:
                    if x2 <= x_passage:
                        distance_x = x1 - x2
                    else:
                        distance_x = x1 + x_limit - x2
                else:
                    if x2 <= x_passage:
                        distance_x = x1 + x2
                    else:
                        distance_x = x1 - x_passage + x2 - x_passage
        ############
    
    # Total distance
    distance = distance_x * _PALLET_WIDTH + distance_y * (_PALLET_LENGTH * 2 + _CORRIDOR_WIDTH)

    return (distance)

In [13]:
def dispatch(layout_number):
    # Create dispatcher
    _TRIPS.sort_values(by=['Order_ID'])

    dispatcher_df = _TRIPS.sort_values(by=['Order_ID'])
    dispatcher_df = dispatcher_df.groupby(by=['Order_ID']).sum()
    max_pallets = 11
    merged_df = _ORDERS.merge(dispatcher_df, how = 'left', left_on = 'Order_ID', right_on = 'Order_ID')

    # Turn values to float
    merged_df['Pallets'] = merged_df['Pallets'].astype(float)
    merged_df['Arrival_Day'] = merged_df['Arrival_Day'].astype(int)


    # Create dispatch lie df
    data = {
        'Line' : [],
        'Day' : [],
        'Orders' : []
    }
    dispatch_line_df = pd.DataFrame(data)

    for day in range(merged_df['Arrival_Day'].min(), merged_df['Arrival_Day'].max() + 1):

        # Select only ordesrs from current day
        day_orders_df = merged_df.loc[merged_df['Arrival_Day'] == day]

        # Order by n pallets and then by shipping hour
        temp_df_ascending = day_orders_df.sort_values(by=['Pallets', 'Shipping_Hour'], ascending = [True, True])

        temp_df_descending = day_orders_df.sort_values(by=['Pallets', 'Shipping_Hour'], ascending = [False, False])

        temp_df_descending = temp_df_descending.reset_index(drop=True)
        temp_df_ascending = temp_df_ascending.reset_index(drop=True)


        dispatch_line = 1
        running = True
        order_index = 0
        orders = []
        last_descending_order_added = -1


        while running:

            # Start new line if there are orders left
            if len(temp_df_ascending) != 0:
                # add pallets form min pallets order (at the top of the df)
                order = temp_df_ascending.iloc[0]
                n_pallets_added = order['Pallets']

                shipping_time = order['Shipping_Hour']

                index = 0
                while index < len(temp_df_descending):
                    row = temp_df_descending.loc[index]


                    if n_pallets_added < max_pallets and row['Shipping_Hour'] != shipping_time:

                        if row['Pallets'] < max_pallets - n_pallets_added:
                            n_pallets_to_add = row['Pallets']

                        else:
                            n_pallets_to_add = max_pallets - n_pallets_added
                            temp_df_descending.at[index,'Pallets'] = row['Pallets'] - n_pallets_to_add
                            index += 1


                        # add palets together
                        n_pallets_added += n_pallets_to_add
                        order_id = row['Order_ID']

                        # add order to array
                        orders.append(order_id)

                        # drop added orders
                        try:
                            temp_df_ascending.drop(temp_df_descending[temp_df_descending['Order_ID'] == order_id].index, inplace = True)
                            temp_df_descending.drop(index, inplace = True)
                            temp_df_descending = temp_df_descending.reset_index(drop=True)
                            temp_df_ascending = temp_df_ascending.reset_index(drop=True)
                        except:
                            pass
                    else: 
                        index += 1

                try:
                    temp_df_ascending.drop(temp_df_ascending[temp_df_ascending['Order_ID'] == order['Order_ID']].index, inplace = True)
                    temp_df_descending.drop(temp_df_descending[temp_df_descending['Order_ID'] == order['Order_ID']].index, inplace = True)
                    temp_df_descending = temp_df_descending.reset_index(drop=True)
                    temp_df_ascending = temp_df_ascending.reset_index(drop=True)
                except:
                    pass

                line = {'Line' : dispatch_line,'Day' : day,'Orders' : orders, 'Total Pallets' : n_pallets_added}
                dispatch_line += 1
                orders = []
                dispatch_line_df = dispatch_line_df.append(line, ignore_index=True)
            else:
                running = False


    dispatch_line_df.to_excel(f'dispatcher{layout_number}.xlsx')
    
    return dispatch_line_df

dispatch_line_df = dispatch('Test')

In [16]:
##########################################

FIRST_LAYOUT_TO_RUN = 10
LAST_LAYOUT_TO_RUN = 10

#########################################

# Create Results Dataframe

data = {'Result' : [
    'Pallets Necessary',
'Trips Necessary',
'Distance Traveled (m)',
'Picker Cost (€)',
'Forklift Picker Cost (€)',
'Pallet Setup Time (h)',
'Travelling Time (h)',
'Picking Time (h)',
'Stabilize Time (h)',
'Despicking Time (h)',
'Repleneshing Time (h)',
'Repleneshing Cost (Forklift)',
'Transportation Cost',
'Calculating Time (s)',
'Total Cost']}
_RESULTS = pd.DataFrame(data)
_RESULTS = _RESULTS.set_index('Result')

data = {'Order' : [], 'Picked_SKU' : [],'Weight' : [], 'X' : [], 'Y' :[], 'Pallet' : [], 'Trip' : []}
_PICKED_DF = pd.DataFrame(data)

# Create Order Trip Dataframe
data = {'Order_ID' : [], 'Trip' : [], 'Pallets' : [],'Time(h)' : []}
_TRIPS = pd.DataFrame(data)

#########################################
# Calculator
#########################################

print('#########################################')
print('Starting Calculator')
print('#########################################')
print('')

# Layout specific costs
for layout_number in range(FIRST_LAYOUT_TO_RUN, LAST_LAYOUT_TO_RUN + 1):

    ##########################################
    # Organize Data Tables
    #########################################
    
    start = timetracker.time()
    print('#########################################')
    print(f'FETCHING LAYOUT {layout_number}')
    print('#########################################')
    print('')
    
    _LAYOUT = _GSH.generateDataframe(_WORKBOOK, f'Layout_{layout_number}')
    index = _LAYOUT_ASSUMPTIONS.index[_LAYOUT_ASSUMPTIONS['Layout'] == str(layout_number)].tolist()
    index = index[0]
    _X_LIMIT = _LAYOUT_ASSUMPTIONS.at[index, 'X_Limit']
    _X_MIDDLE = _LAYOUT_ASSUMPTIONS.at[index, 'X_Middle']

    # Merge ORDERS_SKUS with LAYOUT
    _NEW_ORDERS_SKUS = _ORDERS_SKUS.merge(_LAYOUT, how = 'left', left_on = 'SKU_ID', right_on = 'SKU_ID')
    _NEW_ORDERS_SKUS = _NEW_ORDERS_SKUS.merge(_SKUS, how = 'left', left_on = 'SKU_ID', right_on = 'SKU_ID')
    _NEW_ORDERS_SKUS = _NEW_ORDERS_SKUS[['Order_ID', 'SKU_ID','BU', 'Qty', 'Weight', 'Volume', 'X', 'Y','Z','Cell_ID']]

    # Turn values to int
    _NEW_ORDERS_SKUS['X'] = _NEW_ORDERS_SKUS['X'].astype(int)
    _NEW_ORDERS_SKUS['Y'] = _NEW_ORDERS_SKUS['Y'].astype(int)
    _NEW_ORDERS_SKUS['Qty'] = _NEW_ORDERS_SKUS['Qty'].astype(int)
    _NEW_ORDERS_SKUS['Volume'] = _NEW_ORDERS_SKUS['Volume'].astype(float)
    _NEW_ORDERS_SKUS['Weight'] = _NEW_ORDERS_SKUS['Weight'].astype(float)

    # Get df sorted by Y and sorted by X (ascending on odd y, descending on even y)
    df_to_sort = _NEW_ORDERS_SKUS.sort_values(by = ['Order_ID','Y','X'], ascending = [True, True, True] ,ignore_index = True)

    # Sort odd Y by X ascending
    odd_y_df = df_to_sort.loc[ df_to_sort['Y'] / 2 != df_to_sort['Y'] // 2].sort_values(by = ['Order_ID','Y','X', 'Cell_ID'], ascending = [True, True, True, True] ,ignore_index = True)

    # Sort even Y by X descending
    even_y_df = df_to_sort.loc[ df_to_sort['Y'] / 2 == df_to_sort['Y'] // 2].sort_values(by = ['Order_ID','Y','X','Cell_ID'], ascending = [True, True, False, True] ,ignore_index = True)

    # Join odd with even and sort by Y
    _SORTED_DF = odd_y_df.append(even_y_df).reset_index()
    _SORTED_DF = _SORTED_DF.sort_values(by = ['Order_ID','Y','index'], ascending = [True, True, True] ,ignore_index = True)
    _SORTED_DF = _SORTED_DF.drop(columns=['index'])

    _MAX_PALLETS_PER_TRIP = 2

    t_pallets = 0
    t_trips = 0
    t_distance = 0
    t_picker_cost = float(0)
    t_forklift_picker_cost = float(0)
    t_setup_time = 0
    t_travelling_time = 0
    t_picking_time = 0
    t_stabilize_time = 0
    t_despicking_time = 0
    

    # Repleneshing Cost - Constant throughout layouts
    print('Repleneshing costs...', end='\r')
    repleneshing_cost = 0

    for index, sku in _SKUS.iterrows():
        if sku['SKU_ID'] != '':
            time_spent_repl = 0
            pallet_qty = int(sku['N_Pallets_Needed'])
            if pallet_qty > 1:
                # Froklift up, down and repleneshing
                time_spent_repl += int(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Forklift Time'].iloc[0]['Value']) * 2 * (int(sku['N_Pallets_Needed']) - 1) # Forklift Up and Down
                if _LAYOUT.at[_LAYOUT[_LAYOUT['SKU_ID'] == sku['SKU_ID']].index[0],'PickByBox'] == '1':
                    time_spent_repl += int(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Box Replenish Time'].iloc[0]['Value']) * (int(sku['N_Pallets_Needed']) - 1)
                else:
                    time_spent_repl += int(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Pallet Replenish Time'].iloc[0]['Value']) * (int(sku['N_Pallets_Needed']) - 1)

                time_spent_repl = time_spent_repl / 60 # Mins to hours

                # Forklift Operator moving to postion and back 
                cell1 = '0' #initial position
                cell2 =  str(_LAYOUT.loc[_LAYOUT['SKU_ID'] == sku['SKU_ID']].iloc[0]['Cell_ID']) #sku position
                distance = distance_calculator(cell1, cell2) * 2 / 1000 #back and forth and in km
                time_spent_repl = distance * int(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Picker Speed'].iloc[0]['Value'])
                repleneshing_cost += time_spent_repl * int(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Forklift Operator Cost'].iloc[0]['Value'])

    print(f'Replenshing Costs (€): {int(repleneshing_cost)}')
    print('#########################################')
    print('')

    
    
    print("", end = "\r")
    for i, order in _ORDERS.iterrows():
        
        TEMP_DF = _SORTED_DF.loc[_SORTED_DF['Order_ID'] == order['Order_ID']]
        
        PALLET_DF = TEMP_DF.iloc[0:0]

        pallet_volume = float(0)
        current_pallets = 1
        total_pallets = 1
        trips = 1
        new_trip = False
        count = 0
        distance = float(0)
        picking_time = float(0)
        picker_waiting_time_min = float(0)
        stabilize_time = 0
        despicking_time = 0
        forklift_picking_time_min = 0

        picker_cost = float(0)
        forklift_picker_cost = float(0)
        forklift_distance_come = float(0)
        forklift_distance_goback = float(0)

        index = 0
        order_trip_number = 0
        trip_time_h = 0
        this_trip_pallets = 1
        trip_distance = 0
        while index < len(TEMP_DF):
            
            # Insert picking order
            
            picking_time += _PICKING_TIME
            trip_time_h += _PICKING_TIME / 3600

            row = TEMP_DF.iloc[index]
            
            #Insert picked sku to df
            picked_sku = {'Order' : order['Order_ID'], 'Picked_SKU' : row['SKU_ID'] ,'Weight' : row['Weight'], 'X' : row['X'], 'Y' : row['Y'], 'Pallet' : total_pallets, 'Trip' : trips}
            _PICKED_DF = _PICKED_DF.append(picked_sku, ignore_index=True)

            # If operator starts new trip
            if index == 0 or new_trip == True:
                this_distance = distance_calculator('0', row['Cell_ID'])
                distance += this_distance
                trip_distance += this_distance
                trip_time_h += (this_distance / 1000) / _PICKER_SPEED
                this_distance = 0
                new_trip = False
            else:
                this_distance = distance_calculator(TEMP_DF.iloc[index - 1]['Cell_ID'], row['Cell_ID'])
                distance += this_distance
                trip_distance += this_distance
                trip_time_h += (this_distance / 1000) / _PICKER_SPEED
                this_distance = 0
                
                # Check for weight adjustment
                current_sku = TEMP_DF.iloc[index]['SKU_ID']
                last_sku =  TEMP_DF.iloc[index - 1]['SKU_ID']
                weight = float(_SKUS.loc[_SKUS['SKU_ID'] == current_sku].iloc[0]['Weight'])
                
                if float(_SKUS.loc[_SKUS['SKU_ID'] == current_sku].iloc[0]['Weight']) > float(_SKUS.loc[_SKUS['SKU_ID'] == last_sku].iloc[0]['Weight']):
                    stabilize_time += _STABILIZE_TIME
                    trip_time_h += float(_STABILIZE_TIME) / 60
                    
                else:
                    pass

            qty_to_place = 0
            
            # Check if we have to get another pallet
            if pallet_volume + row['Qty'] * row['Volume'] > _MAX_PALLET_VOLUME:
                vol_left = _MAX_PALLET_VOLUME - pallet_volume
                qty_to_place = vol_left // row['Volume'] # divisão interia 5 // 2 = 2
                
                TEMP_DF.at[TEMP_DF[TEMP_DF['SKU_ID'] == current_sku].index[0],'Qty'] = row['Qty'] - qty_to_place
                total_pallets += 1
                pallet_volume = 0
                
                # Check if we need another trip
                if current_pallets == 2:
                    new_trip = True
                    trips += 1
                    current_pallets = 1
                    this_trip_pallets = 2

                else:
                    current_pallets += 1
                    this_trip_pallets = 1
                    new_trip = False
            else:
                new_trip = False
                qty_to_place = row['Qty']
                pallet_volume += (qty_to_place * row['Volume'])

            # Add qty to pallet
            PALLET_DF = PALLET_DF.append(row)
            PALLET_DF.iloc[len(PALLET_DF)-1]['Qty'] = qty_to_place
            
            
            # Add forklift costs if sku is in  level 1
            if  row['Z'] == 1 or row['Z'] == '1':
                # calculate distance from 0 to cell 
                forklift_distance_come += distance_calculator('0',row['Cell_ID'])
                # calculate forklift time picking (up and down)
                forklift_picking_time_min += _FORKLIFT_TIME_UPDOWN * 2
                # picker waits for forklift down
                picker_waiting_time_min += _FORKLIFT_TIME_UPDOWN
                trip_time_h += _FORKLIFT_TIME_UPDOWN / 60
                # forklift waits for picker to pick
                forklift_picking_time_min += float(_PICKING_TIME) / 60
                # forklift goes back
                forklift_distance_goback += distance_calculator(row['Cell_ID'], '0')
            else:
                pass

            # Go back to inital point and calculate despicking time
            if new_trip == True or index == len(TEMP_DF) - 1:
                this_distance = distance_calculator(row['Cell_ID'], '0')
                distance += this_distance
                trip_distance += this_distance
                trip_time_h += (this_distance / 1000) / _PICKER_SPEED
                # add trip to _TRIPS
                order_trip_number += 1
                new_trip = {'Order_ID' : order['Order_ID'], 'Trip' : order_trip_number, 'Trip Distance' : trip_distance,'Pallets' : this_trip_pallets,'Time(h)' : trip_time_h}
                _TRIPS = _TRIPS.append(new_trip, ignore_index=True)
                
                trip_time_h = 0
                this_trip_pallets = 1
                trip_distance = 0

                # Order Pallet by weight and BU
                PALLET_DF = PALLET_DF.sort_values(by = ['Weight','BU'], ascending = [False, True] ,ignore_index = True)

                # Initialize BU counter
                BUs = ''
                previous_BU = ''
                n_mixed_BUs = []

                for j, sku in PALLET_DF.iterrows():
                    # How many mixed BUs?
                    BU = sku['BU']
                    if len(BUs) < 1: 
                        BUs = BUs + BU
                    elif BU != previous_BU:
                        if BU in BUs:
                            n_mixed_BUs.append(len(BUs) - BUs.index(BU)) #n mixed bus = n bus in between
                        BUs = BUs + BU
                    else:
                        pass

                    previous_BU = BU

                # Calculate despicking costs
                mixed_BUs = max(n_mixed_BUs) if len(n_mixed_BUs) > 0 else 0
                despicking_bu_time = 0
                if len(BUs) == 1: # Only 1 BU
                    despicking_bu_time += 0
                elif mixed_BUs == 0: # 2+ BUs but divided
                    despicking_bu_time += 1.5 * PALLET_DF['Qty'].sum()
                elif mixed_BUs == 2: 
                    despicking_bu_time += 4.6 * PALLET_DF['Qty'].sum()
                elif mixed_BUs == 3:
                    despicking_bu_time += 6.7 * PALLET_DF['Qty'].sum()
                elif mixed_BUs >= 4:
                    despicking_bu_time += 10.2 * PALLET_DF['Qty'].sum()

                despicking_time += despicking_bu_time
                PALLET_DF = PALLET_DF.iloc[0:0]

                if index == len(TEMP_DF) - 1:
                    index +=1
            else:
                index += 1

            count += 1


        # dsicpicking cost
        picker_cost = 0
        despicking_time = float(despicking_time) / 3600
        picker_cost += despicking_time * _PICKER_COST

        # distance cost
        travelling_time = (distance / 1000)/ _PICKER_SPEED
        picker_cost += travelling_time  * _PICKER_COST

        # picking cost
        picking_time = picking_time / 3600
        picker_cost += picking_time * _PICKER_COST
        
        # picker waiting cost
        picker_waiting_time = picker_waiting_time_min / 60
        picker_cost += picker_waiting_time * _PICKER_COST

        # pallet setup cost
        setup_time = total_pallets * _PICKING_SETUP_TIME
        setup_time = setup_time / 60
        forklift_picker_cost += setup_time * _FORKLIFT_COST

        # stabilize cost
        stabilize_time = stabilize_time / 60
        picker_cost += stabilize_time * _PICKER_COST
        
        # picker waits for forklift guy
        waiting_time = float(forklift_distance_come) / 1000 * _PICKER_SPEED
        picker_cost += waiting_time * _PICKER_COST
        
        #forklift cost
        forklift_time = (forklift_distance_come + forklift_distance_goback) / 1000 * _PICKER_SPEED
        forklift_time += forklift_picking_time_min /60
        forklift_picker_cost += forklift_time * _FORKLIFT_COST
        
        total_time_h = stabilize_time + setup_time + picking_time + despicking_time + travelling_time

        t_pallets += total_pallets
        t_trips += trips
        t_distance += distance
        t_picker_cost += picker_cost
        t_setup_time += setup_time
        t_travelling_time += travelling_time
        t_picking_time += picking_time
        t_stabilize_time += stabilize_time
        t_despicking_time += despicking_time
        t_forklift_picker_cost += forklift_picker_cost

        print(f'Order {i + 1} of {len(_ORDERS)} | Time Elapsed (min): {round((timetracker.time() - start)/60, 2)}', end="\r")

    print("")


    #Transportation Cost
    t_transportation_cost =  _TRANSPORTATION_COST * t_pallets
    end = timetracker.time()


    print(f'Pallets Necessary: {t_pallets}')
    print(f'Trips Necessary: {t_trips}')
    print(f'Distance Traveled (m): {t_distance}')
    print(f'Picker Cost (€): {t_picker_cost}')
    print(f'Forklift Picker Cost (€): {t_forklift_picker_cost}')
    print(f'Pallet Setup Time: {t_setup_time}')
    print(f'Travelling Time: {t_travelling_time}')
    print(f'Picking Time: {t_picking_time}')
    print(f'Stabilize Time: {t_stabilize_time}')
    print(f'Despicking Time: {t_despicking_time}')
    print(f'Repleneshing Cost (Forklift): {repleneshing_cost}')
    print(f'Transportation Cost: {t_transportation_cost}')
    print(f'Calculating Time (s): {end - start}')
    print('')
    print(f'Total Cost: {repleneshing_cost + t_picker_cost + t_transportation_cost}')
    print('')
    print('#########################################')
    print('')
    
    results = {
    'Pallets Necessary' : t_pallets,
    'Trips Necessary': t_trips,
    'Distance Traveled (m)': t_distance,
    'Picker Cost (€)': t_picker_cost,
    'Forklift Picker Cost (€)': t_forklift_picker_cost,
    'Pallet Setup Time (h)': t_setup_time,
    'Travelling Time (h)': t_travelling_time,
    'Picking Time (h)': t_picking_time,
    'Stabilize Time (h)':t_stabilize_time,
    'Despicking Time (h)':t_despicking_time,
    'Repleneshing Time (h)' : time_spent_repl,
    'Repleneshing Cost (Forklift)':repleneshing_cost,
    'Transportation Cost': t_transportation_cost,
    'Calculating Time (s)': round((end - start)/60,2),
    'Total Cost' : repleneshing_cost + t_picker_cost + t_transportation_cost
    }
    
    _RESULTS[f'Layout_{layout_number}'] = results.values()
    results = {}
    
    dispatch_line_df = dispatch(layout_number)
    
    _TRIPS.to_excel(f'trips{layout_number}.xlsx')
    _TRIPS = _TRIPS.iloc[0:0]
        
    
_RESULTS.to_excel('results.xlsx')

#########################################
Starting Calculator
#########################################

#########################################
FETCHING LAYOUT 10
#########################################

Replenshing Costs (€): 11521
#########################################

Order 1120 of 1120 | Time Elapsed (min): 43.34
Pallets Necessary: 5663
Trips Necessary: 3112
Distance Traveled (m): 1324361.7199999983
Picker Cost (€): 21989.57954444444
Forklift Picker Cost (€): 7098.065366666665
Pallet Setup Time: 283.15000000000003
Travelling Time: 220.726953333333
Picking Time: 1236.145833333332
Stabilize Time: 68.78333333333367
Despicking Time: 573.7493611111104
Repleneshing Cost (Forklift): 11521.222200000006
Transportation Cost: 45304.0
Calculating Time (s): 2598.1461288928986

Total Cost: 78814.80174444444

#########################################



# Pick By Box Cell Creation

In [314]:
# Find n_cells needed with pick by box
_LAYOUT = _GSH.generateDataframe(_WORKBOOK, 'Layout_7')
# Turn values to float
_LAYOUT['WidthNecessary'] = _LAYOUT['WidthNecessary'].astype(float)
_LAYOUT['PickByBox'] = _LAYOUT['PickByBox'].astype(int)

new_df = _LAYOUT
pallet_width = 0.8


cell = 0
current_cell_width_occupied = 0
current_level = 1
cell_last_z = 0
for index, row in new_df.iterrows():
    
    if index == 0: last_row = row
        
    if row['Z'] == '0':

        # Se pick by box seguido
        if last_row['PickByBox'] == row['PickByBox'] and row['PickByBox'] == 1:
            if current_cell_width_occupied + row['WidthNecessary'] > pallet_width:
                wasted_width += pallet_width - current_cell_width_occupied
                current_cell_width_occupied = 0

                # Move ot 2nd height
                if current_level == 1:
                    # cell is same but goes to level 2
                    current_level = 2

                # change cell if height 1 adn 2 are filled
                else:
                    cell += 1
                    current_level = 1
            else: 
                current_cell_width_occupied += row['WidthNecessary']
        else:
            cell += 1
    else:
        if cell_last_z != cell : # Se não há nenhum Z nesta célula deixo ficar este Z nesta célula
            pass
        else: # senão passao para a a seguir
            cell += 1
                
                
            
    new_df.at[index,'Cell'] = cell

cells = new_df['Cell'].max()
racks = cells / 3
new_df.to_excel('cells_needed.xlsx')
print(racks)
new_df